Installation and Importing Libraries

The first part of code involves installing necessary Python packages and importing libraries

In [ ]:
!pip install anvil-uplink


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=15c4fee0a3013aac962cb975ff759b7c4977cf9bab7adaad063309af1d5c65d4
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


This part of the code installs the anvil-uplink package and sets up a connection to an Anvil server. Anvil is a platform for building full-stack web apps with Python. The anvil.server.connect() function establishes a connection to the Anvil server using a unique Uplink key.

In [ ]:
import anvil.server

anvil.server.connect("server_FJ4UUTRGY7FFDI7JTWSY6HDR-GQBPEB2ADKGAB4MD")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Public link to branch master" as SERVER


In [ ]:
!pip3 install python-espncricinfo

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 5.3 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=0cab844aeb1a87d042a6a5ab810e1214a3dcd7b13527d259a8106fdbd3524ec5
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [159]:
import requests
import xml.etree.ElementTree as ET
import re
from datetime import datetime
import pandas as pd
from difflib import get_close_matches
import anvil.server
import pickle
import anvil.media
from sklearn.preprocessing import StandardScaler

# Load mapping data from CSV files
df_venue = pd.read_csv('/content/drive/MyDrive/DATA602-FP/venue_mappings.csv')
df_tossWinner = pd.read_csv('/content/drive/MyDrive/DATA602-FP/toss_winner_mappings.csv')
df_team = pd.read_csv('/content/drive/MyDrive/DATA602-FP/team_mappings.csv')

file_path = "/content/drive/MyDrive/DATA602-FP/final_df.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

X = df.drop('result', axis=1)

# Get the label of the first column
first_column = X.columns[0]

# Drop the first column
X = X.drop(first_column, axis=1)


# Create a StandardScaler object
scaler = StandardScaler()

scaler.fit(X)


# This function takes a label and a DataFrame as input. It finds the best match for the label in the DataFrame's 'Label' column.
# If a match is found, it returns the corresponding class value. If no exact match is found, it returns the class value of the closest match.
# This function is useful for encoding categorical variables based on their similarity to known labels. It uses the get_close_matches function from the
# difflib library to find the closest matches. The cutoff parameter in the get_close_matches function determines how strict the matching should be. A cutoff
# of 0.0 means that it will return the closest match regardless of how similar it is.


def encoder(lable, df):
    # Find the best match in the label column
    matches = get_close_matches(lable, df['Label'].tolist(), n=1)
    if matches:
        # If a match is found, return the corresponding class value
        return df[df['Label'] == matches[0]]['Class'].values[0]
    else:
        # If no match is found, return the closest match
        return df[df['Label'] == get_close_matches(lable, df['Label'].tolist(), n=1, cutoff=0.0)[0]]['Class'].values[0]



# This function takes an XML string as input, parses it to extract cricket match data, and returns a list of matches.
# Each match is represented as a dictionary with keys corresponding to different match attributes
# This function is used to parse cricket match data from an XML string. It extracts various match attributes such as the teams, current score, gender,
# match ID, and more. It also fetches additional match data from the ESPN Cricinfo API. The function is designed to process Twenty20 matches and
# calculates the current run rate and required run rate based on the innings number. The parsed match data is returned as a list of dictionarie
# This part of the function determines the toss winner and the batting and bowling teams based on the toss decision and innings number. It then encodes
# the venue, batting team, bowling team, and toss winner using the encoder function. The match is then appended to the matches list. If there are no matches,
# the function returns a message indicating this. Otherwise, it returns the list of matches. Please ensure to replace the XML string with your own when using
# this function. This function is useful for parsing cricket match data from an XML string and preparing it for further analysis or machine learning tasks.
# It is designed to process Twenty20 matches and takes into account various match attributes such as the teams, current score, gender, match ID, and more.
# It also fetches additional match data from the ESPN Cricinfo API and calculates the current run rate and required run rate based on the innings number.
# The parsed match data is returned as a list of dictionaries.


def parse_cricinfo(xml_string):
    root = ET.fromstring(xml_string)
    matches = []

    for item in root.findall('.//item'):
        match = {}
        match['description'] = item.find('description').text

        team_l, team_r = match['description'].split(' v ')
        current_score = 0
        if '*' in team_l:
          match['current_score'] = team_l.split('*')[0].split()[-1]
          current_score=match['current_score']
          target_runs=team_r.split('*')[0].split()[-1]
        elif '*' in team_r:
          match['current_score'] = team_r.split('*')[0].split()[-1]
          current_score=match['current_score']
          target_runs=team_l.split('*')[0].split()[-1]




        title = item.find('title').text
        if 'Women' in title:
          match['gender'] = 0
        else:
          match['gender'] = 1




        link = item.find('link').text
        match_id = re.search('match/(.*).html', link)
        if match_id:
            match_id = match_id.group(1)
            match['match_id'] = match_id
            guid = item.find('guid').text
            match['guid'] = guid

            # Fetch match data
            url = "https://www.espncricinfo.com/matches/engine/match/{0}.json".format(str(match_id))
            r = requests.get(url)
            match_data = r.json()['match']


            if match_data.get('general_class_card') == 'Twenty20':
                # Check if the match has started
                start_datetime_gmt = datetime.strptime(match_data.get('start_datetime_gmt'), '%Y-%m-%d %H:%M:%S')
                if start_datetime_gmt > datetime.utcnow():
                    continue

                # Check if the match has a winner
                if match_data.get('winner_team_id') != '0':
                    continue


                try:
                   current_runs, wickets_lost = current_score.split('/')
                   match['current_runs'] = current_runs
                   match['wickets_lost'] = wickets_lost
                except ValueError:
                   current_runs=current_score
                   match['current_runs']=current_runs
                   match['wickets_lost']=0

                match['live_overs_unique'] = match_data.get('live_overs_unique')
                over, ball = map(float, match['live_overs_unique'].split('.'))
                match['crr'] = float(current_runs) / (over*6 + ball)

                match['live_innings_number'] = match_data.get('live_innings_number')
                if match['live_innings_number']== "1":
                  match['target_runs']=0
                  match['rrr'] = 0
                else:
                  FirstInn_runs, FirstInnWickets_lost = target_runs.split('/')
                  match['target_runs']= FirstInn_runs
                  match['rrr'] = ((float(FirstInn_runs)-float(current_runs)+1)/(120 - (over * 6 + ball)))
                match['team1_name'] = match_data.get('team1_short_name')
                match['team2_name'] = match_data.get('team2_short_name')
                match['team1_id'] = match_data.get('team1_id')
                match['team2_id'] = match_data.get('team2_id')
                match['venue'] = match_data.get('ground_name')
                match['toss_dec'] = 0 if match_data.get('toss_decision') == "1" else 1


                if match_data.get('toss_winner_team_id') == match['team1_id']:
                    match['toss_winner_team_name'] = match['team1_name']
                    if match['toss_dec']==0:
                      if match['live_innings_number']=="1":
                        match['batting_team']=match['team1_name']
                        match['bowling_team']=match['team2_name']
                      else:
                        match['batting_team']=match['team2_name']
                        match['bowling_team']=match['team1_name']
                    else:
                      if match['live_innings_number']=="1":
                        match['batting_team']=match['team2_name']
                        match['bowling_team']=match['team1_name']
                      else:
                        match['batting_team']=match['team1_name']
                        match['bowling_team']=match['team2_name']
                else:
                    match['toss_winner_team_name'] = match['team2_name']
                    if match['toss_dec']==1:
                      if match['live_innings_number']=="1":
                        match['batting_team']=match['team1_name']
                        match['bowling_team']=match['team2_name']
                      else:
                        match['batting_team']=match['team2_name']
                        match['bowling_team']=match['team1_name']
                    else:
                      if match['live_innings_number']=="1":
                        match['batting_team']=match['team2_name']
                        match['bowling_team']=match['team1_name']
                      else:
                        match['batting_team']=match['team1_name']
                        match['bowling_team']=match['team2_name']

                match['venue_class'] = encoder(match['venue'], df_venue)
                match['batting_class'] = encoder(match['batting_team'], df_team)
                match['bowling_class'] = encoder(match['bowling_team'], df_team)
                match['tossWinner_class'] = encoder(match['toss_winner_team_name'], df_team)

                matches.append(match)
                if matches == []:
                  return "There are no ongoing live matches"

    return matches




# This function takes a match dictionary as input and returns two dictionaries: 'data' and 'info'.
# The 'data' dictionary contains encoded match attributes for further analysis or machine learning tasks.
# The 'info' dictionary contains additional match information
# This function is used to prepare cricket match data for further analysis or machine learning tasks. It takes a match dictionary as input and returns two
# dictionaries: ‘data’ and ‘info’. The ‘data’ dictionary contains encoded match attributes such as the gender, innings number, batting team, bowling team,
# venue, toss winner, toss decision, wickets lost, current run rate, and required run rate. The ‘info’ dictionary contains additional match information such
#  as the description, batting team, and bowling team.




def create_data_dict(match):
    data = {
        'gender_encoded': int(match['gender']),
        'inning_no': int(match['live_innings_number']),
        'batting_team_encoded': int(match['batting_class']),
        'bowling_team_encoded': int(match['bowling_class']),
        'venue_encoded': int(match['venue_class']),
        'toss_winner_encoded': int(match['tossWinner_class']),
        'toss_decision_encoded': int(match['toss_dec']),
        'wickets_lost': int(match['wickets_lost']),
        'crr': float(match['crr']),
        'rrr': float(match['rrr'])
    }
    info ={
        'description': match['description'],
        'bat_team': match['batting_team'],
        'bowling_team': match['bowling_team']
    }
    return data,info



# This function uses a trained model to predict the result of ongoing live cricket matches.
# It fetches live match data from the ESPN Cricinfo API, preprocesses the data, and uses the model to predict the win probabilities of the teams
# This function is used to predict the result of ongoing live cricket matches. It fetches live match data from the ESPN Cricinfo API, parses the XML
#  string to extract match data, preprocesses the match data, and uses a trained model to predict the win probabilities of the teams. The function returns
#   a list of strings, each representing the predicted result of a match



@anvil.server.callable
def predict_result():
    # Load the trained model
    with open('/content/drive/MyDrive/DATA602-FP/xgb_model.pkl', 'rb') as f:
        model = pickle.load(f)

    url = "http://static.cricinfo.com/rss/livescores.xml"
    response = requests.get(url)
    xml_string = response.text


    matches = parse_cricinfo(xml_string)
    if not matches:
      return "There are no ongoing live matches"
    else:
      results = []  # List to store the results
      for match in matches:
          data,info = create_data_dict(match)
          print(data)
          df_new = pd.DataFrame([data])
          scaled_df = scaler.transform(df_new)
          result = model.predict_proba(scaled_df)
          second_number = result[0][1] * 100
          rounded_number = round(second_number, 2)
          match_result = info['description'] +" \n The win probability of " + info['bat_team'] + " is "+ str(rounded_number)+"%  \n The win probability of "+info['bowling_team']+" is "+ str(round(100-rounded_number,2))+"%"
          results.append(match_result)  # Add the result to the list
      return results  # Return the list of results







The anvil.server.wait_forever() function is used in Anvil applications to keep the server running indefinitely. This is typically used when you have a script that you want to keep running to wait for incoming client requests.

In [ ]:
anvil.server.wait_forever()

{'gender_encoded': 1, 'inning_no': 2, 'batting_team_encoded': 101, 'bowling_team_encoded': 26, 'venue_encoded': 148, 'toss_winner_encoded': 101, 'toss_decision_encoded': 1, 'wickets_lost': 2, 'crr': 1.6779661016949152, 'rrr': 1.1967213114754098}
{'gender_encoded': 1, 'inning_no': 2, 'batting_team_encoded': 101, 'bowling_team_encoded': 26, 'venue_encoded': 148, 'toss_winner_encoded': 101, 'toss_decision_encoded': 1, 'wickets_lost': 2, 'crr': 1.65, 'rrr': 1.2166666666666666}
{'gender_encoded': 1, 'inning_no': 2, 'batting_team_encoded': 101, 'bowling_team_encoded': 26, 'venue_encoded': 148, 'toss_winner_encoded': 101, 'toss_decision_encoded': 1, 'wickets_lost': 2, 'crr': 1.6666666666666667, 'rrr': 1.2}
{'gender_encoded': 1, 'inning_no': 2, 'batting_team_encoded': 101, 'bowling_team_encoded': 26, 'venue_encoded': 148, 'toss_winner_encoded': 101, 'toss_decision_encoded': 1, 'wickets_lost': 2, 'crr': 1.639344262295082, 'rrr': 1.2203389830508475}
{'gender_encoded': 1, 'inning_no': 2, 'batting